In [9]:
import tensorflow 
import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [19]:
data = pd.read_csv('combined_data.csv')

data.head()

,_id,Initial_Year,Initial_Month,Forecast_Year,Foreast_Month,Province_ID,Province_NameTH,Rainfall(mm),Rainfall.mm.
0,1.0,2021,1,2021,1,10,กรุงเทพมหานคร,5.359580,NaN
1,2.0,2021,1,2021,1,11,จ.สมุทรปราการ,4.724574,NaN
2,3.0,2021,1,2021,1,12,จ.นนทบุรี,6.060826,NaN
3,4.0,2021,1,2021,1,13,จ.ปทุมธานี,6.709093,NaN
4,5.0,2021,1,2021,1,14,จ.พระนครศรีอยุธยา,2.295685,NaN


In [12]:
# Features and Target
X = data[['Initial_Year', 'Initial_Month', 'Forecast_Year', 'Foreast_Month', 'Province_ID']]
y = data['Rainfall(mm)']

# Scale the feature data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

In [13]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)  # No sigmoid because it's a regression problem now
])

# Compile the model
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
optimizer = Adam(learning_rate=0.0001, clipvalue=1.0)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# Train model
roun = int(input("Number of training epochs: "))
batch_siz = int(input("Batch size: "))
train_round = int(input("Number of training rounds for averaging: "))

# Track average rainfall predictions
rainfall_predictions = []

for i in range(train_round):
    print(f"Training iteration {i+1}/{train_round}")
    model.fit(X_train, y_train, epochs=roun, batch_size=batch_siz, verbose=0)
    
    # Save the model after each training iteration
    model.save(f"model_round_{i+1}.h5")

    # Predict for the last day in the dataset
    X_last_day = X.iloc[-1].values
    predicted_rainfall = model.predict(np.array([X_last_day]))[0][0]
    
    rainfall_predictions.append(predicted_rainfall)

# Compute the average rainfall prediction
average_rainfall = sum(rainfall_predictions) / len(rainfall_predictions)
print(f"Average predicted rainfall over {train_round} rounds: {average_rainfall:.2f} mm")

# Custom prediction
iY = int(input("Enter the forecast year: "))
iM = int(input("Enter the forecast month: "))
province_id = int(input("Enter the province ID: "))

# Scale the custom input
custom_input = np.array([[iY, iM, iY, iM, province_id]])
custom_input_scaled = scaler.transform(custom_input)

# Predict rainfall for custom input
predicted_rainfall_custom = model.predict(custom_input_scaled)[0][0]
print(f"Predicted rainfall for year {iY}, month {iM}, province {province_id}: {predicted_rainfall_custom:.2f} mm")

Training iteration 1/3


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Training iteration 2/3


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Training iteration 3/3


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Average predicted rainfall over 3 rounds: -15653.82 mm
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted rainfall for year 2024, month 9, province 10: -25.84 mm


c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [15]:
import urllib
url = 'https://data.hii.or.th/sv/api/3/action/datastore_search?limit=5&resource_id=e45c665d-8389-4bf1-889b-37c42ca200ee&q=title:jones'  
fileobj = urllib.urlopen(url)
print(fileobj.read())

AttributeError: module 'urllib' has no attribute 'urlopen'

In [ ]:
data

In [ ]:
import pandas as pd

# โหลด dataset ของคุณ
data = pd.read_csv('.csv')

# ฟังก์ชันสำหรับคำนวณตาม ID
def calculate_by_id(province_id):
    # คัดกรองข้อมูลตาม ID ที่กรอก
    filtered_data = data[data['Province_ID'] == province_id]
    
    if filtered_data.empty:
        print("ไม่พบข้อมูลสำหรับ Province_ID ที่กรอก")
        return

    # ตัวอย่างการคำนวณ (เปลี่ยนเป็นการคำนวณที่คุณต้องการ)
    total_rainfall = filtered_data['Rainfall(mm)'].sum()
    average_rainfall = filtered_data['Rainfall(mm)'].mean()

    print(f"สำหรับ Province_ID {province_id}:")
    print(f"ปริมาณฝนรวม: {total_rainfall:.2f} มิลลิเมตร")
    print(f"เฉลี่ยฝน: {average_rainfall:.2f} มิลลิเมตร")

# รับ input จากผู้ใช้
province_id_input = int(input("กรุณากรอก Province_ID: "))
calculate_by_id(province_id_input)
